In [ ]:
import numpy as np

# metaheuristics example

# --------------------------------------------------------------------
# Data for illustrative example
# --------------------------------------------------------------------
# no. of jobs
n = 5
# no. of machines
m = 3
# due date for job j
d = [38, 115, 120, 51, 178]
# no. of sublots for job j
s = [2, 2, 3, 3, 3]
# earliness penalty for job j
alpha = [1, 3, 1, 2, 1]
# tardiness penalty for job j
beta = [2, 1, 3, 1, 1]
# sublot processing time for job j on machine i
lt = np.array([[10, 15, 5, 9, 20],
               [9, 10, 3, 12, 15],
               [8, 11, 1, 5, 10]])
# given sequence
pi = [3, 1, 2, 4, 5]

# --------------------------------------------------------------------
# HELPER FUNCTIONS
# --------------------------------------------------------------------
def get_gaps(start_times, end_times):
    return [start_times[-1, i+1] - end_times[-1, i] 
            for i in range(len(start_times[-1]) - 1)]
#gaps = get_gaps(start_times = start_times, end_times = end_times)

   
# --------------------------------------------------------------------
# INPUT FUNCTIONS
# --------------------------------------------------------------------

# sort based on the sequence
def sort_data(sequence, due_dates, sublots, alpha, beta, processing_times):
    """
    sort the given data based on the sequence pi
    
    sequence        : list
    due_dates       : list, sorted as per sequence
    sublots         : list, number of sublots sorted as per sequence
    alpha           : list, earliness penalties sorted as per sequence
    beta            : list, tardiness penalties sorted as per sequence
    processing_times: m * n numpy array, sorted as per sequence
    
    returns [d_sorted, s_sorted, alpha_sorted, beta_sorted, lt_sorted]
    """
    
    k = [1,2,3,4,5]
    
    pi_zip = list(zip(k,sequence))
    pi_sorted = sorted(pi_zip , key = lambda s: s[1] )

    #d sorted
    
    d_zip = list(zip(pi_sorted, due_dates))
    d_sort = sorted(d_zip , key = lambda s: s[0][0] )

    d_sorted=[]
    for i in d_sort:
        d_sorted.append(i[1])

    #s sorted
    
    s_zip = list(zip(pi_sorted, sublots))
    s_sort = sorted(s_zip , key = lambda s: s[0][0] )

    s_sorted=[]
    for i in s_sort:
        s_sorted.append(i[1])

    #alpha sorted
    
    alpha_zip = list(zip(pi_sorted, alpha))
    alpha_sort = sorted(alpha_zip , key = lambda s: s[0][0] )

    alpha_sorted=[]
    for i in alpha_sort:
        alpha_sorted.append(i[1])

    #beta sorted
    
    beta_zip = list(zip(pi_sorted, beta))
    beta_sort = sorted(beta_zip , key = lambda s: s[0][0] )

    beta_sorted=[]
    for i in beta_sort:
        beta_sorted.append(i[1])

    #lt sorted

    lt_shape = processing_times.T
    lt_zip = list(zip(pi_sorted, lt_shape))
    lt_sort = sorted(lt_zip , key = lambda s: s[0][0] )
    lt_blank=[]
    for i in lt_sort:
        lt_blank.append(i[1])

    lt_sorted = np.array(lt_blank).T
    
    
    return [d_sorted, s_sorted, alpha_sorted, beta_sorted, lt_sorted]
    
#d, s, alpha, beta, lt = sort_data(pi)
#d_sorted, s_sorted, alpha_sorted, beta_sorted, lt_sorted = sort_data(pi)
#d, s, alpha, beta, lt = sort_data(pi, d, s, alpha, beta, lt)



def initialize_data(machines, jobs, sublots, processing_times):
    """
    initializes start and end times given the data

    sublots         : list, number of sublots sorted as per sequence
    processing_times: m * n numpy array, sorted as per sequence

    returns [start_times, end_times]
    """
    m = machines
    n = jobs
    lt = processing_times
    start_times = np.array([[0] * n] * m)
    end_times = np.array([[0] * n] * m)

    def build_sublot_times():
        '''
            build sublot start times and end times
        :param m                    : int, job num
        :param n                    : int, sequence num
        :param sublot               : list, sublot processing times
        :param processing_times     : m * n numpy array, sorted as per sequence
        :return                     : sublot_start_times, sublot_end_times
        '''

        sublot_start_times =\
            [[[0 for s in range(sublots[j])] for j in range(n)] for i in range(m)]
        # size: m x n x s

        sublot_end_times =\
            [[[0 for s in range(sublots[j])] for j in range(n)] for i in range(m)]
        # size: m x n x s

        for i in range(m):
            for j in range(n):
                for s in range(sublots[j]):
                    # choose previous sublot end times
                    if j == 0 and s == 0:  # first sublot, set as 0
                        Y_pre_sublot = 0

                    else:  # not first sublot
                        last_i = i  # the same machine
                        if s == 0:  # consider the same machine prevoius job's prevoius sublot (j-1, s[-1])
                            last_j = j - 1
                            last_s = sublots[j-1] - 1

                        else:  # s >= 1 # consider the same machine current job's previous sublot (s-1)
                            last_j = j
                            last_s = s - 1

                        Y_pre_sublot = sublot_end_times[last_i][last_j][last_s]

                    # choose previous machine end times
                    if i == 0:  # first machine, has no previous
                        Y_pre_machine = 0
                    else:
                        Y_pre_machine = sublot_end_times[i-1][j][s]

                    # define Y_list
                    Y_list = [Y_pre_machine, Y_pre_sublot]

                    start_time = max(Y_list)

                    sublot_start_times[i][j][s] = start_time  # X
                    sublot_end_times[i][j][s] = start_time + lt[i][j]  # Y

                    
                   

        return [sublot_start_times, sublot_end_times]


    sub_start, sub_end = build_sublot_times()

    from functools import reduce
    import operator

    start_times = []
    for mac in range(m):
        start_times.append(reduce(operator.concat, sub_start[mac]))

    end_times = []
    for mac in range(m):
        end_times.append(reduce(operator.concat, sub_end[mac]))


    return [np.array(start_times), np.array(end_times)]

#start_times, end_times = initialize_data(machines = m,
#                                         jobs = n,
#                                         sublots = s,
#                                         alpha = alpha,
#                                         beta = beta,
#                                         processing_times = lt)

    

def get_last_idxs(jobs, sublots): 
   
    n = jobs
    s = sublots

    last_idx = []
    for i in range(n):
        last_idx.append(sum(s[:i+1])-1)

    return last_idx
#last_idx = get_last_idxs(jobs = n, sublots = s)

# completion times
def get_C(end_times, last_idx):
   
    C = []
    for i in range(len(last_idx)):
        C.append(end_times[-1,last_idx[i]])
    return C
#C = get_C(end_times = end_times, 
#          last_idx = last_idx)

# earliness
def get_E(end_times, last_idx, jobs, due_dates):

    n = jobs
    E = []
    for i in range(n):
        E.append(max(0,due_dates[i] - end_times[-1,last_idx[i]]))
    return E

# tardiness
def get_T(end_times, last_idx, jobs, due_dates):

    n = jobs
    T = []
    for i in range(n):
        T.append(max(0, end_times[-1,last_idx[i]] - due_dates[i]))
    return T
#T = get_T(end_times = end_times, 
#          last_idx = last_idx, 
#          jobs = n, 
#          due_dates = d)

# I(k)
def get_I(start_times, end_times, last_idx):
   
    gap_list = []
    for i in range(len(start_times[-1])-1):
        gap_list.append(start_times[-1,i+1]-end_times[-1,i])
    
    add = 0
    I = []

    gap_list.append(0)
    
    for j in range(len(gap_list)):

        if j in last_idx:
            I.append(add)
            add = 0
            continue
        else:
            add += gap_list[j]
    return I
#I = get_I(start_times = start_times,
#          end_times = end_times,
#          jobs = n,
#          last_idx = last_idx)

# I'(k)
def get_Iprime(start_times, end_times, last_idx):

    gap_list = []
    for i in range(len(start_times[-1])-1):
        gap_list.append(start_times[-1,i+1]-end_times[-1,i])
    
    add = 0
    Iprime = [None]

    for j in range(len(gap_list)):

        if j in last_idx:

            add += gap_list[j]
            Iprime.append(add)
            add = 0
            
        else:
            continue
    return Iprime
#Iprime = get_Iprime(start_times = start_times,
#                    end_times = end_times,
#                    jobs = n,
#                    last_idx = last_idx)


# --------------------------------------------------------------------
# Objective function
# --------------------------------------------------------------------
def objective(alpha, beta, E, T):
    return sum(a * e + b * t for a, b, e, t in 
               zip(alpha, beta, E, T))


# --------------------------------------------------------------------
# Additional functions
# --------------------------------------------------------------------
def UB_init(n):
    return [0] * n

def get_UB(E, k, alpha, beta):
    if E[k] > 0:    return alpha[k]
    else:           return -beta[k]

def get_UNB(E, k, r, alpha, beta):
#    return sum(UB[i : i + r])
    return sum(get_UB(E, i, alpha, beta) for i in range(k, k+r+1))

def move(k, start_times, end_times, last_idx, time_units):
    start_times[-1, last_idx[k]] += time_units
    end_times[-1, last_idx[k]] += time_units
    for i in range(len(start_times[-1]) - 1):
        if end_times[-1, i] > start_times[-1, i+1]:
            end_times[-1, i+1] += (end_times[-1, i] - start_times[-1, i+1])
            start_times[-1, i+1] = end_times[-1, i]
    return [start_times, end_times]
           
def update(start_times, end_times, last_idx, jobs, due_dates):
    return [get_gaps(start_times, end_times), 
            get_C(end_times, last_idx), 
            get_I(start_times, end_times, last_idx),
            get_Iprime(start_times, end_times, last_idx), 
            get_E(end_times, last_idx, jobs, due_dates), 
            get_T(end_times, last_idx, jobs, due_dates)]


# --------------------------------------------------------------------
# net benefit of movement algorithm
# --------------------------------------------------------------------
def solve_nbm(jobs, start_times, end_times, due_dates, last_idx, E, T, I, Iprime,
              alpha, beta):
    """
    Implements the net benefit of movement algorithm to solve
    a flowshop scheduling problem for a given sequence of jobs
    
    inputs:
        jobs       : int
        start_times: m * n array
        end_times  : m * n array
        due_dates  : list
        last_idx   : list
        E          : list
        T          : list
        I          : list
        Iprime     : list
        alpha      : list
        beta       : list
    """
    
    n = jobs
    d = due_dates
    UB = UB_init(n)
    UNB = UB_init(n)

    k = n - 1    
    while k >= 0:
    
        # step 2
        if k == n - 1:
#            print("k = {}, E[k] = {}".format(k, E[k]))
            if E[k] > 0:
                start_times[-1, last_idx[k]] += E[k]
                end_times[-1, last_idx[k]] += E[k]
                gaps, C, I, Iprime, E, T = update(start_times = start_times, 
                                                  end_times = end_times, 
                                                  last_idx = last_idx, 
                                                  jobs = n, 
                                                  due_dates = d)
            k -= 1
            
        # step 3
        else:
            # find smallest r: I(k+r+1) or I'(k+r+1) > 0
            # i.e., if both are zero, increment r
            r = 0
            while k+r+1 < n-1 and I[k+r+1] == 0 and Iprime[k+r+1] == 0:
                r += 1
            # step 4
#            print(k)
#            print(len(UB))
            UB[k] = get_UB(E, k, alpha, beta)
            UNB[k] = get_UNB(E, k, r, alpha, beta)
            
            # step 5
#            print("k = {}, UNB[k] = {}".format(k, UNB[k]))
            if UNB[k] > 0:
                A = I[k+r+1] + Iprime[k+r+1]  # i.e. I(k+r+1) + I'(k+r+1)
    #            B = min(E[k : k+r+1])         # i.e. E(k) to E(k+r)
                B = min([E[x] for x in range(k, k+r+1) if E[x] > 0])
                M = min(A, B)
                start_times, end_times = move(k, start_times, end_times, last_idx, M)
                gaps, C, I, Iprime, E, T = update(start_times, end_times, last_idx, n, d)
    
            # step 6    
            else:
                k -= 1
                if k >= 0:
                    continue
                else:
                    break

    gaps, C, I, Iprime, E, T = update(start_times, end_times, last_idx, n, d)

    return [start_times, end_times, C, E, T, I, Iprime]
    
#solve_nbm(jobs = n, 
#          start_times = start_times, 
#          end_times = end_times, 
#          due_dates = d, 
#          last_idx = last_idx, 
#          E = E, 
#          T = T, 
#          I = I, 
#          Iprime = Iprime)
          
# --------------------------------------------------------------------
# step 7
def get_e(jobs, E, T, due_dates, processing_times):
    n, d, lt = jobs, due_dates, processing_times
    E, T = E, T
    e = [0] * n
    for k in range(n):
        if E[k] > 0:
            e[k] = d[k] - lt[-1, k] - E[k]
        else:
            e[k] = d[k] - lt[-1, k] + T[k]
    return e
#e = get_e(jobs = n, E = E, T = T, due_dates = d, processing_times = lt)



# --------------------------------------------------------------------
# step 7 continued
# Delay processing on other sublots on each machine without
# affecting e(k) for all k

def delay_processing(jobs, machines, sublots, start_times, end_times, last_idx):
    """
    Given optimal schedule, delays processing of 
    all (except last) sublots for all machines
    
    inputs:
        jobs       : int
        machines   : int
        sublots    : list
        start_times: m * n array
        end_times  : m * n array
        due_dates  : list
        last_idx   : list
        
    returns: [start_times, end_times]
    
    """
    
    n = jobs
    m = machines
    s = sublots
    
    length = sum(s)
    idxs = [idx for idx in range(length) if idx not in last_idx]
    
    for mch in range(m-1, -1, -1):
        for i in range(length-1, -1, -1):
            
            # last machine
            if mch == m - 1:
                # don't touch last sublots
                if i in idxs:
                    move = start_times[mch, i+1] - end_times[mch, i]
                    start_times[mch, i] += move
                    end_times[mch, i] += move
                
            else:
                if i == length - 1:
                    move = start_times[mch+1, i] - end_times[mch, i]
                    start_times[mch, i] += move
                    end_times[mch, i] += move
                else:
                    new_end = min(start_times[mch, i+1], start_times[mch+1, i])
                    move = new_end - end_times[mch, i]
                    start_times[mch, i] += move
                    end_times[mch, i] += move
            
    return [start_times, end_times]
    
#start_times, end_times = delay_processing(jobs = n, 
#                                          machines = m, 
#                                          sublots = s, 
#                                          start_times = start_times, 
#                                          end_times = end_times, 
#                                          last_idx = last_idx)


# --------------------------------------------------------------------
# NBM example problem
# --------------------------------------------------------------------
def initialize_nbm(n, m, d, s, alpha, beta, lt, pi):
    """
    Solves example problem
    
    inputs:
        n = 5
        m = 3
        d = [38, 115, 120, 51, 178]
        s = [2, 2, 3, 3, 3]
        alpha = [1, 3, 1, 2, 1]
        beta = [2, 1, 3, 1, 1]
        lt = np.array([[10, 15, 5, 9, 20],
                       [9, 10, 3, 12, 15],
                       [8, 11, 1, 5, 10]])
        pi = [3, 1, 2, 4, 5]
        
    returns: [start_times, end_times, d, s, alpha, beta, lt, last_idx, 
              gaps, C, E, T, I, Iprime]
    """
    
    # USES INPUT FUNCTION FROM ABOVE    
    # sort given data according to sequence pi
    d, s, alpha, beta, lt = sort_data(pi, d, s, alpha, beta, lt)
    
    # USES INPUT FUNCTION FROM ABOVE
    # initialize start_times, end_times
    start_times, end_times = initialize_data(m, n, s, lt)

    # USES INPUT FUNCTIONS FROM ABOVE    
    # get initial gaps and last indices for all jobs
    gaps = get_gaps(start_times, end_times)
    last_idx = get_last_idxs(n, s)
   
    # USES INPUT FUNCTIONS FROM ABOVE    
    # calculate initial completion times, earliness, tardiness, I and Iprime
    C = get_C(end_times, last_idx)
    E = get_E(end_times, last_idx, n, d)
    T = get_T(end_times, last_idx, n, d)
    I = get_I(start_times, end_times, last_idx)
    Iprime = get_Iprime(start_times, end_times, last_idx)

    # EXECUTE NBM ALGORITHM
    # get the optimal schedule
    start_times, end_times, C, E, T, I, Iprime = \
    solve_nbm(n, start_times, end_times, d, last_idx, E, T, I, Iprime, alpha, beta)
    z = objective(alpha, beta, E, T)
    gaps = get_gaps(start_times, end_times)

    # compute optimal starting times of the last sublots of jobs on the last machine
    e = get_e(n, E, T, d, lt)

    # delay processing to get the final schedule
#    start_times, end_times = delay_processing(n, m, s, start_times, end_times, last_idx)
                                              
    # return all results
    return [start_times, end_times, d, s, alpha, beta, lt, last_idx, 
             gaps, C, E, T, I, Iprime, z, e]

# solves nbm
Result = initialize_nbm(n,m,d,s,alpha,beta,lt,pi)
Result

[array([[  0,   5,  10,  15,  25,  35,  50,  65,  74,  83,  92, 112, 132],
        [  5,  10,  15,  25,  35,  50,  65,  75,  87,  99, 112, 132, 152],
        [  8,  13,  35,  36,  44,  60, 104, 115, 120, 125, 130, 147, 168]]),
 array([[  5,  10,  15,  25,  35,  50,  65,  74,  83,  92, 112, 132, 152],
        [  8,  13,  18,  34,  44,  60,  75,  87,  99, 111, 127, 147, 167],
        [  9,  14,  36,  44,  52,  71, 115, 120, 125, 130, 140, 157, 178]]),
 [120, 38, 115, 51, 178],
 [3, 2, 2, 3, 3],
 [1, 1, 3, 2, 1],
 [3, 2, 1, 1, 1],
 array([[ 5, 10, 15,  9, 20],
        [ 3,  9, 10, 12, 15],
        [ 1,  8, 11,  5, 10]]),
 [2, 4, 6, 9, 12],
 [4, 21, 0, 0, 8, 33, 0, 0, 0, 0, 7, 11],
 [36, 52, 115, 130, 178],
 [84, 0, 0, 0, 0],
 [0, 14, 0, 79, 0],
 [25, 0, 33, 0, 18],
 [None, 0, 8, 0, 0],
 191,
 [35, 44, 104, 125, 168]]

In [ ]:
pip install pandas


In [ ]:
the_output = Result[9:14]
the_output

[[36, 52, 115, 130, 178],
 [84, 0, 0, 0, 0],
 [0, 14, 0, 79, 0],
 [25, 0, 33, 0, 18],
 [None, 0, 8, 0, 0]]

In [ ]:
import pandas as pd
dfObj = pd.DataFrame(the_output, columns = ['1' , '2', '3', '4', '5'], index=['C(k)', 'E(k)', 'T(k)', 'I(k)', 'Iprime(k)'])

print(f'f(pi) = {Result[-2]}')
dfObj

f(pi) = 191


,1,2,3,4,5
C(k),36.0,52,115,130,178
E(k),84.0,0,0,0,0
T(k),0.0,14,0,79,0
I(k),25.0,0,33,0,18
Iprime(k),NaN,0,8,0,0


In [ ]:
e_result = Result[-1]

df_e = pd.DataFrame(e_result, columns = ['e(k)'], index=['1' , '2', '3', '4', '5'])
df_e

,e(k)
1,35
2,44
3,104
4,125
5,168
